## Homework 04.1
#### Category: computational

Let us consider the following boundary value problem: find $u: I = (0, 1) \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- u'' = 1, & x \in I,\\
u(0) = 0,\\
u(1) = 0.
\end{cases}
\end{equation*}

We have seen during the tutorial how to solve this problem using FEniCS. The goal of this exercise is for you to implement a finite element method for this problem, without using FEniCS.

1. Define a mesh of the interval $I$ dividing it in $n$ equispaced cells. Store the vertices of the mesh in a `numpy` array, generated using the [`np.linspace`](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html) function. You may use $n = 30$; double check whether you should be providing $n$ or $n + 1$ as the third argument to `np.linspace`.

2. Let $i = 0, \dots, n$ be the index of the $i$-th node $x_i$. Write on paper the expression of the "hat" functions $\varphi_i(x)$, $i = 0, \dots, n$, which define the Lagrange basis functions for the space of linear polynomials $\mathbb{P}_1$. Implement the "hat" functions in Python, and draw a plot using `matplotlib`.

3. Write on paper, and then implement & draw in Python, the expression of the first derivative $\varphi_i'(x)$ of the hat function $\varphi_i(x)$, $i = 0, \dots, n$.

4. Exploiting the expressions that you have written on paper, determine the formula for the $i$-th element of the right-hand side vector of the finite element system, i.e.
$$ b_i = F(\varphi_i), \quad i = 0, \dots, n,$$
where the linear functional associated to this problem is
$$ F(v) = \int_I v  \ \mathrm{d}x.$$

    *Hint*: you have to split integration over $I$ as the sum of integrals over each cell $(x_{k-1}, x_k)$, and (due to the local support of the hat functions) determine which values of $k$ make the integrand non-zero for the provided index $i$.

5. Exploiting the expressions that you have written on paper, determine the formula for the $(i, j)$-th element of the stiffness matrix, i.e.
$$ A_{ij} = a(\varphi_j, \varphi_i), \quad i,j = 0, \dots, n,$$
where the bilinear form associated to this problem is
$$ a(u, v) = \int_I u' \ v' \ \mathrm{d}x.$$

    *Hint*: you have to split integration over $I$ as the sum of integrals over each cell $(x_{k-1}, x_k)$, and (due to the local support of the hat functions) determine which values of $k$ make the integrand non-zero for the provided pair $(i, j)$.
    
6. The solution $\mathbf{u}$ of the finite element problem can thus be obtained as solution to the linear system
$$ A \mathbf{u} = \mathbf{b}. $$
Impose the Dirichlet boundary conditions by modifying the matrix $A$ and the right-hand side vector $\mathbf{b}$ so that:
    * $u(0) = 0$, i.e. $u_0 = 0$,
    * $u(1) = 0$, i.e. $u_n = 0$.

   One way to do this is to clear the first (associated to $u(0)$) and last (associated to $u(1)$) rows & columns of $A$:
   $$ A_{:0} = A_{0:} = A_{:n} = A_{n:} = 0,$$
   put a diagonal entry equal to 1
   $$ A_{00} = A_{nn} = 1,$$
   and change the first and last entry of the right-hand side vector $\mathbf{b}$ so that they store the value of the boundary condition
   $$b_0 = u(0) \equiv 0, \qquad b_n = u(1) \equiv 0.$$

7. After imposing the boundary conditions, solve the linear system using [`np.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

8. Plot the finite element solution, as well as the exact solution $$u(x) = \frac{1}{2} x (1 - x).$$

9. Plot the error between the finite element solution and the exact solution. Compute the maximum absolute value of the error.
   
   *Hint*: in order to evaluate the error at a generic location $x$, you will need to define the function
   $$u_h(x) = \sum_{i = 0}^n u_i \varphi_i(x),$$
   where $u_i$ is the $i$-th entry of the solution vector $\mathbf{u}$, and you already have a Python implementation for each "hat" function $\varphi_i(x)$.

In [ ]:
# 1 defining the mesh 
# I = (0, 1) -- domain
import numpy as np
n = 30                        # number of elements of the mesh
num_vertices = n + 1
xi = np.linspace(0,1,num_vertices)
if len(xi) == num_vertices:
  print("Ok! The number the mesh's elements is equal to %i"%n)
else:
  print("Pay attention!You must control the number of intervals" )

Ok! The number the mesh's elements is equal to 30


The third argument of linspace must be (n+1)


In [ ]:
# 2 defining the basis of P1 linear function space
import numpy as np

class hat_func(object):
  """ Class of hat functions
      that are the basis of Lagrange 
      function space of order 1

  """
  def __init__(self, i: int, dx: float):
    self.i = i
    self.dx = dx
    self.xi = dx*i
  def __call__(self, x: np.ndarray) -> np.ndarray:
    xi = self.xi
    return 0*(x < xi-dx) + (x - (xi-dx))/dx * np.logical_and(xi-dx <= x,  x < xi) + (1 - (x - xi)/dx)*np.logical_and( xi <=  x, x < (xi + dx)) + 0*(x >= (xi + dx))

In [ ]:
# 3 plotting the hat functions using matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
n = 30
fig, axs = plt.subplots(n + 1, figsize=(150,150))

dx = 1./n
x = np.linspace(0,1,10000)

for i in range(n+1):
    y = hat_func(i, dx)
    axs[i].set_title('%i' % (i))
    axs[i].plot(x, y(x))

In [ ]:
# 3 writing the first derivatives of the hat functions
class diff_hat_func(object):
  """ Class of derivatives
      of the Lagrange basis functions
      of order 1
  """
  def __init__(self, i: int, dx: float):
    self.i = i
    self.dx = dx
    self.xi = dx*i
  def __call__(self, x: np.ndarray) -> np.ndarray:
    xi = self.xi
    return 0*(x < xi-dx) + 1./dx* np.logical_and(xi-dx <= x,  x < xi) -1./dx*np.logical_and( xi <=  x, x < (xi + dx)) + 0*(x >= (xi + dx))  

In [ ]:
# plotta 
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 4
import numpy as np
n = 30
fig, axs = plt.subplots(n+1, figsize=(250,250))

dx = 1./30
x = np.linspace(0,1,10000)

for i in range(n+1):
    y = diff_hat_func(i, dx)
    axs[i].set_title('%i' % (i))
    axs[i].plot(x, y(x))

$$b_i = F(\phi_i) \hspace{0.2 cm} i = 0, 1, ..., n$$
$$F(v) = \int_{I} u dx$$
$$b_i = F(\phi_i) = \int_{I} \phi_i dx \hspace{0.2 cm} i=0, 1, 2, ... , n$$
$$b_0 = F(\phi_0) = \int_{I} \phi_0 dx$$
$$b_1 = F(\phi_1) = \int_{I} \phi_1 dx$$
$$...$$

Splitting integrals
$$b_0 = F(\phi_0) = \int_{0}^{1} \phi_0 dx = \int_{x_0}^{x_1} \frac{x - x_0}{dx} dx$$
$$b_1 = F(\phi_1) = \int_{0}^{1} \phi_1 dx = \int_{x_0}^{x_1} \frac{x - x_0}{dx} dx + \int_{x_1}^{x_2} \frac{1 - (x - x_1)}{dx} dx$$
$$b_2 = F(\phi_2) = \int_{0}^{1} \phi_2 dx = \int_{x_1}^{x_2} \frac{x - x_1}{dx} dx + \int_{x_2}^{x_3} \frac{1 - (x - x_2)}{dx} dx$$
$$...$$
where:
$$dx = x_i - x_{i-1}$$
or
$$dx = x_{i+1} - x_i$$


$$b_i = \int_{x_{i-1}}^{x_i} \frac{x - x_{i-1}}{dx} dx + \int_{x_i}^{x_{i+1}} \frac{1 - (x - x_i)}{dx} dx$$

In [ ]:
# introduce a formula to calculate integrals
# with the trapezoidal rule
def trapezoidal(f, a, b, n):
    h = (b-a)/float(n)
    I = f(a) + f(b)
    for k in range(1, n, 1):
        x = a + k*h
        I += 2*f(x)
    I *= h/2
    return I

In [ ]:
# 4 determine the right-hand side vector bi
In = 1000
Ia = 0.
Ib = 1.
b = []
for i in range(n+1):
    dx = 1. / n
    x = np.linspace(0,1,100)
    y = hat_func(i, dx)
    b.append(trapezoidal(y,Ia,Ib,In))

In [ ]:
# converting b from a list to a numpy vector
b = np.asarray(b)
np.shape(b)

(31,)

bilinear form
$$a(u,v) = \int_{I} u' v' dx$$
$$A_{ij} = a(\phi_j, \phi_i) = \int_{I} \phi'_j \phi'_i dx$$


 determine the stiffness matrix
$$A_{ij} = a(\phi_j, \phi_i) = \int_{I} (\phi'_j , \phi'_i) dx$$
$\phi'_i = $\begin{cases}
                0 \hspace{0.4 cm} x < x_{i-1} \\
                1/dx \hspace{0.4 cm} x_{i-1} < x < x_i \\
                -1/dx \hspace{0.4 cm} x_i < x < x_{i+1} \\
                0 \hspace{0.4 cm} x > x_{i+1} \\
            \end{cases}

dim($V_h$) = 11 $\rightarrow$ $ A \in Mat(11\times 11)$
$$A_{00} = a(\phi_0, \phi_0) = \int_{I} \phi'_0 \phi'_0 dx = \int_{x_0}^{x_1} \frac{1}{dx} * \frac{1}{dx} dx = 100 * [x]^{1/0.33...}_0 = 30$$
$$A_{01} = a(\phi_0, \phi_1) = \int_{I} \phi'_0 \phi'_1 dx = \int_{x_0}^{x_1} \frac{1}{dx} * \frac{-1}{dx}  dx = -100 * [x]^{1/0.33...}_0 = -30$$
$$...$$
where
$$dx = 10/30 = 0.33...$$

In [ ]:
# stiffness matrix
A = np.zeros((n+1,n+1))
for i in range(n+1):
  for j in range(n+1):
    y = diff_hat_func(i,dx)
    y1 = diff_hat_func(j,dx)
    prodotto = lambda x: y(x)*y1(x)
    A[i][j] = trapezoidal(prodotto, 0,1.,1000)
print(A)

In [ ]:
# 6 imposing Dirichlet BC
A[0, :] = 0
A[:, 0] = 0
A[n, :] = 0.
A[:, n] = 0.
A[0, 0] = 1.
A[n, n] = 1.
print("Sriffness matrix A after imposing boundary conditions: \n", A)

In [ ]:
# imposing boundary conditions 
# on right-hand side vector b
b[0] = 0.
b[n] = 0.
print("Right-hand side vector b after imposing bc: ", b)

In [ ]:
# 7 solving the linear system using numpy linalg solve
from numpy.linalg import solve
u = solve(A,b)

In [ ]:
# 8 determine the exact solution and plot with fem solution
def exact_sol(x):
    return 0.5*x*(1-x) 

In [ ]:
uex = exact_sol(xi)
print("The exact solution is: ", uex)

In [ ]:
np.shape(uex)

In [ ]:
print(u)
print(xi)

In [ ]:
fig = plt.figure(figsize=(20, 20))
plt.plot(xi,u,'ro--', xi, uex, 'b--')


In [ ]:
np.allclose(np.dot(A, u), b)

In [ ]:
# 9 plot the error between the fem solution and the exact solution 
eh_max = np.max(np.abs(u - uex))
print("The maximum error between fem solution and exact solution is: \n", eh_max)

## Homework 04.2
#### Category: methodological

Let us consider the following boundary value problem: find $u: I = (0, 1) \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- u'' = 1, & x \in I,\\
u(0) = 0,\\
u(1) = 0.
\end{cases}
\end{equation*}

In this course we discuss a discretization by the finite element method. However, other discretization methods are possible. The goal of this exercise is to highlight that, for this very specific example, there is a close relationship between the finite element method and a finite difference method.

1. Define a mesh of the interval $I$ dividing it in $n$ equispaced cells (i.e., $n + 1$ nodes). Let $i = 0, \dots, n$ be the index of the $i$-th node $x_i$, and $h = x_{i + 1} - x_{i} = \dots = x_1 - x_0$. Consider the two following finite difference approximations of the first derivative $u'$ of a function $u$ at the node $x_i$:
\begin{equation*}
\begin{aligned}
u'(x_i) &\approx \delta_{h, i}^+ u := \frac{u(x_{i + 1}) - u(x_i)}{h},\\
u'(x_i) &\approx \delta_{h, i}^- u := \frac{u(x_{i}) - u(x_{i - 1})}{h},\\
\end{aligned}
\end{equation*}

   You have already seen such finite difference approximations applied to ordinary differential equations in the numerical analysis course that you have taken during your bachelor degree when discussing the forward and backward (explicit and implicit, respectively) Euler methods.
 
   Show that these two approximations can be combined to provide a finite difference approximation of the second derivative $u''$ of a function $u$ at the node $x_i$:
$$u''(x_i) \approx \delta_{h, i}^+ (u') \approx \delta_{h,i}^+ (\delta_{h,i}^- u) := \frac{u(x_{i + 1}) - 2 u(x_i) + u(x_{i - 1})}{h^2}$$

2. Collect the finite difference unknowns in the vector
$$\mathbf{u} = [u_0, u_1, \dots, u_i, \dots, u_{n-1}, u_n]^T := [u(x_0), u(x_1), \dots, u(x_i), \dots, u(x_{n-1}), u(x_n)]^T.$$

   Show that the finite difference approximation to the solution of the boundary value problem is given by the following recurrence relation
\begin{equation*}
\begin{cases}
\frac{- u_{i + 1} + 2 u_i - u_{i - 1}}{h^2} = 1, & i = 1, \dots, n - 1,\\
u_0 = 0,\\
u_n = 0.
\end{cases}
\end{equation*}

3. Write the recurrence relation as a linear system
$$ A \mathbf{u} = \mathbf{b}. $$
Recognize that (up to a multiplication by a factor $h$) the finite difference linear system is the same one that you obtained in Homework 04.1, item 6. This shows that the finite difference method and the first order Lagrange finite element scheme coincide for this specific boundary value problem.

domanda 1
$$u''(x_i) \sim \delta_{h,i}^{+}(\delta_{h,i}^{-} u) = \frac{\delta_{h+1}^{-}u(x_{i+1}) - \delta_{h+1}^{-} u(x_i)}{h} = \frac{u(x_{i+1} - u(x_i) - u(x_i) - u(x_{i-1})}{h} = \frac{u(x_{i+1}) - 2u(x_i) - u(x_{i-1})}{h}$$

## Homework 04.3
#### Category: computational
#### Note: this exercise can be the starting point for an optional exam project. Contact us towards the end of the course if you are interested.

Let us consider the following boundary value problem: find $u: I = (0, 1) \to \mathbb{R}$ such that
\begin{equation*}
\begin{cases}
- \varepsilon u'' + u' = 0, & x \in I,\\
u(0) = 0,\\
u(1) = 1,
\end{cases}
\end{equation*}
where $\varepsilon < 1$.

Use FEniCS to solve the boundary value problem on a mesh made of 15 equispaced intervals, and with $\mathbb{P}_1$ finite elements. Consider three values of $\varepsilon$, namely $\varepsilon = 1$, $\varepsilon = 10^{-1}$, $\varepsilon = 10^{-2}$. Does the solution look correct when $\varepsilon = 10^{-2}$? Change the number of intervals and/or the degree of the finite element space until you get a more accurate solution.

*Hint*: in order to avoid the error `This integral is missing an integration domain.`, which arises in `ufl` when one of the factors in the multiplication is zero, in this specific case you should define the linear functional as `F = dolfinx.fem.Constant(mesh, 0.0) * vh * dx` rather than `F = 0 * vh * dx`.

In [ ]:
try:
    import dolfinx
except ImportError:
    !wget "https://github.com/fem-on-colab/fem-on-colab.github.io/raw/0f2a84c/releases/fenicsx-install-real.sh" -O "/tmp/fenicsx-install.sh" && bash "/tmp/fenicsx-install.sh"
    import dolfinx

In [ ]:
try:
    import multiphenicsx
except ImportError:
    !pip3 install "multiphenicsx@git+https://github.com/multiphenics/multiphenicsx.git@906a91b"
    import multiphenicsx

In [ ]:
import dolfinx.fem
import dolfinx.mesh
import mpi4py
import multiphenicsx.io
import numpy as np
import petsc4py
import ufl

In [ ]:
# 1 create and plot the mesh 
# I = (0, 1) -- domain
mesh = dolfinx.mesh.create_unit_interval(mpi4py.MPI.COMM_WORLD, 15)

In [ ]:
multiphenicsx.io.plot_mesh(mesh)

In [ ]:
# define vertices
points = mesh.geometry.x
print("Points are  3D vector: \n",points)

In [ ]:
# create the adjacency list 
connectivity_cells_to_vertices = mesh.topology.connectivity(mesh.topology.dim, 0)
print("connectivity_cells_to_vertices: \n", connectivity_cells_to_vertices)

In [ ]:
# determine the number of elements of the mesh
num_cells = len(connectivity_cells_to_vertices)
print("Number of cells: \n", num_cells)

In [ ]:
# check the correctness of the mesh
for c in range(num_cells):
  # Print the ID of the current cell
  print("Cell ID", c, "is defined by the following vertices:")
  # Print the vertices of the current cell
  for v in connectivity_cells_to_vertices.links(c):
    #pass
    print("\t" + "Vertex ID", v, "is located at x =", points[v][0])
    left_boundary_entities = dolfinx.mesh.locate_entities_boundary(
        mesh, mesh.topology.dim - 1, lambda x: np.isclose(x[0], 0.0))
    print(left_boundary_entities)
    right_boundary_entities = dolfinx.mesh.locate_entities_boundary(
        mesh, mesh.topology.dim - 1, lambda x: np.isclose(x[0], 1.0))
    print(right_boundary_entities)

In [ ]:
# 3 define the FEM FunctionSpace
# as Langrange of order 1
Vh = dolfinx.fem.FunctionSpace(mesh, ("P", 1))
Vh_dim = Vh.dofmap.index_map.size_local

# solving the problem for epsilon = 1.
epsilon = 1.
uh = ufl.TrialFunction(Vh)
vh = ufl.TestFunction(Vh)
dx = ufl.dx
uh_prime = uh.dx(0)
vh_prime = vh.dx(0)

# bilinear form
a = epsilon * uh_prime * vh_prime * dx + uh_prime * vh * dx
# linear functional
F = dolfinx.fem.Constant(mesh, 0.0) * vh * dx

# determine the degrees of freedom of the boundaries
left_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim - 1, left_boundary_entities)
right_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim - 1, right_boundary_entities)

# determine the constant function of boundary conditions
zero = dolfinx.fem.Constant(mesh, 0.)
one = dolfinx.fem.Constant(mesh, 1.)

In [ ]:
# applying boundary conditions
bcs = [dolfinx.fem.dirichletbc(zero, left_boundary_dofs, Vh), dolfinx.fem.dirichletbc(one, right_boundary_dofs, Vh)]

# determine the solution
solution = dolfinx.fem.Function(Vh)
problem = dolfinx.fem.petsc.LinearProblem(
        a, F, bcs=bcs, u=solution,
        petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()

In [ ]:
# plot the solution
multiphenicsx.io.plot_scalar_field(solution, "u_h")

In [ ]:
# determine the exact solution
xyz = ufl.SpatialCoordinate(mesh)
x = xyz[0]

from math import e
u_ex = e**x / (e - 1) + 1./(1 - e)

In [ ]:
# determine the error
eh_squared_ufl = (u_ex - solution)**2 * dx
eh_squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(eh_squared_ufl))

print("Error for epsilon = 1: ", eh_squared)
#print("Errore per epsilon = 0.1: ", eh_squared)
#print("Errore per epsilon = 0.01: ", eh_squared)

In [ ]:
# solving the problem for epsilon = 0.1 
epsilon = 0.1
uh = ufl.TrialFunction(Vh)
vh = ufl.TestFunction(Vh)
dx = ufl.dx

uh_prime = uh.dx(0)
vh_prime = vh.dx(0)

a = epsilon * uh_prime * vh_prime * dx + uh_prime * vh * dx
F = dolfinx.fem.Constant(mesh, 0.0) * vh * dx

left_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim - 1, left_boundary_entities)
right_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim - 1, right_boundary_entities)

zero = dolfinx.fem.Constant(mesh, 0.)
one = dolfinx.fem.Constant(mesh, 1.)

In [ ]:
bcs = [dolfinx.fem.dirichletbc(zero, left_boundary_dofs, Vh), dolfinx.fem.dirichletbc(one, right_boundary_dofs, Vh)]
solution = dolfinx.fem.Function(Vh)


problem = dolfinx.fem.petsc.LinearProblem(
        a, F, bcs=bcs, u=solution,
        petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()


In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "u_h")

In [ ]:
xyz = ufl.SpatialCoordinate(mesh)
x = xyz[0]

from math import e
u_ex = e**x / (e - 1) + 1./(1 - e)

In [ ]:
eh_squared_ufl = (u_ex - solution)**2 * dx


eh_squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(eh_squared_ufl))

#print("Errore per epsilon = 1: ", eh_squared)
print("Error for epsilon = 0.1: ", eh_squared)
#print("Errore per epsilon = 0.01: ", eh_squared)


In [ ]:
# solving the problem for epsilon = 0.01
epsilon = 0.01
uh = ufl.TrialFunction(Vh)
vh = ufl.TestFunction(Vh)

dx = ufl.dx

uh_prime = uh.dx(0)
vh_prime = vh.dx(0)
a = epsilon * uh_prime * vh_prime * dx + uh_prime * vh * dx

F = dolfinx.fem.Constant(mesh, 0.0) * vh * dx

left_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim - 1, left_boundary_entities)

right_boundary_dofs = dolfinx.fem.locate_dofs_topological(Vh, mesh.topology.dim - 1, right_boundary_entities)

zero = dolfinx.fem.Constant(mesh, 0.)
one = dolfinx.fem.Constant(mesh, 1.)

In [ ]:
bcs = [dolfinx.fem.dirichletbc(zero, left_boundary_dofs, Vh), dolfinx.fem.dirichletbc(one, right_boundary_dofs, Vh)]

solution = dolfinx.fem.Function(Vh)

problem = dolfinx.fem.petsc.LinearProblem(
        a, F, bcs=bcs, u=solution,
        petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
_ = problem.solve()


In [ ]:
multiphenicsx.io.plot_scalar_field(solution, "u_h")

In [ ]:
xyz = ufl.SpatialCoordinate(mesh)
x = xyz[0]

from math import e
u_ex = e**x / (e - 1) + 1./(1 - e)

In [ ]:
eh_squared_ufl = (u_ex - solution)**2 * dx

eh_squared = dolfinx.fem.assemble_scalar(dolfinx.fem.form(eh_squared_ufl))

#print("Errore per epsilon = 1: ", eh_squared)
#print("Errore per epsilon = 0.1: ", eh_squared)
print("Error for epsilon = 0.01: ", eh_squared)
